**Part 1 (50 points):**

  Implement the Viterbi algorithm for HMMs for Eisner's Ice Cream Problem (predict whether each day is hot or cold based on the number of ice creams eaten).  See eisner.hmm.xls for a bit more discussion on the problem. Remember that the Viterbi algorithm computes the most likely sequence for an input.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
observation_url = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/observationProbs.csv'
transition_url = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/transitionProbs.csv'
test_url = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/testData.csv'
observation_df = pd.read_csv(observation_url)
transition_df = pd.read_csv(transition_url)
test_df = pd.read_csv(test_url)

In [ ]:
display(observation_df.head(), transition_df.head(), test_df.head())

,P(x|...),C,H
0,1,0.6407,0.0002
1,2,0.1481,0.5341
2,3,0.2122,0.4657


,P(x|...),C,H,START
0,C,0.87,0.47,0.5
1,H,0.07,0.47,0.5
2,STOP,0.06,0.06,0.0


,SeqNumber,Obs1,Obs2,Obs3,Obs4,Obs5
0,1,2,3,3,2,3
1,2,2,3,2,2,0
2,3,3,1,3,3,1
3,4,2,1,1,0,0
4,5,1,1,1,2,3


In [ ]:
h_or_c = np.array(transition_df)[:2,0]
observations = np.array(observation_df)[:,0]
observation_seq = np.array(test_df)[:, 1:6]
#display(h_or_c, observations, observation_seq)

**Initialize all the data structure that is going to be needed for viterbi:**

In [ ]:
def initilize_data(h_or_c, obs):
  prior_prob = {}
  stop_prob = {}
  transition_prob = {}
  emission_prob = {}
  for i in range(len(h_or_c)):
    weather = h_or_c[i]
    # Setup the dictionary with prior probobility given the correspond weather
    prior_prob[weather] = np.array(transition_df[['START']])[i][0]
    # Setup the dictionary with stop probobility given the correspond weather
    stop_prob[weather] = np.array(transition_df)[2][i+1]
    # Initilize two empty inner dictionary for transition_prob and emissioin_prob
    prob_today_given_yesterday = {}
    observation_prob_given_weather = {}
    for j in range(len(h_or_c)):
        #Setup the dictionary with the probobility of today's weather given
        #yesterday's weather
        prob_today_given_yesterday[h_or_c[j]] = np.array(transition_df)[i][j+1]
    transition_prob[weather] = prob_today_given_yesterday
    for k in range(len(obs)):
        #Setup the dictionary with the probobility of observation given
        #the weather cold or hot
        observation_prob_given_weather[obs[k]] = np.array(observation_df)[:,i+1][k]
    emission_prob[weather] = observation_prob_given_weather
  return prior_prob, stop_prob, transition_prob, emission_prob

In [ ]:
prior_prob, stop_prob, transition_prob, emission_prob = initilize_data(h_or_c, observations)
display(prior_prob, stop_prob, transition_prob, emission_prob)

{'C': 0.5, 'H': 0.5}

{'C': 0.06, 'H': 0.06}

{'C': {'C': 0.87, 'H': 0.47}, 'H': {'C': 0.07, 'H': 0.47}}

{'C': {1.0: 0.6407, 2.0: 0.1481, 3.0: 0.2122},
 'H': {1.0: 0.0002, 2.0: 0.5341, 3.0: 0.4657}}

In [ ]:
def viterbi(h_or_c, obs, prior_prob, stop_prob, transition_prob, emission_prob):
  m_table = [{}]
  c_table = {}
  #Calculate the initial m11 m12 and store in the m table
  for weather in h_or_c:
    m_table[0][weather] = prior_prob[weather] * emission_prob[weather][obs[0]]
    c_table[weather] = [weather]
  #Take the farther step to calculate the rest mi1 and mi2
  for i in range(1, len(obs)):
    m_table.append({})
    c_table_temp = {}
    for y_weather in h_or_c:
      prob_t_given_y = {}
      for t_weather in h_or_c:
        prob_t_given_y[t_weather] = m_table[i-1][t_weather] \
                                    * transition_prob[t_weather][y_weather] \
                                    * emission_prob[y_weather][obs[i]]
      # Find the weather with the max probobility
      key_list = list(prob_t_given_y.keys())
      val_list = list(prob_t_given_y.values())
      max_prob = max(val_list) # Find the max probility
      m_table[i][y_weather] = max_prob
      position = val_list.index(max_prob)
      key_weather = key_list[position]
      c_table_temp[y_weather] = c_table[key_weather] + [y_weather]
    c_table = c_table_temp
  #Calculate the last day's probobility
  prob_lastday_given_yesterday = {}
  for weather in h_or_c:
    prob_lastday_given_yesterday[weather] = m_table[len(obs) - 1][weather] * stop_prob[weather]
  key_list = list(prob_lastday_given_yesterday.keys())
  val_list = list(prob_lastday_given_yesterday.values())
  max_prob = max(val_list) # Find the max probility of the last day
  position = val_list.index(max_prob)
  last_day_weather = key_list[position]
  return c_table[last_day_weather]

In [ ]:
i = 1
for obs in observation_seq:
  obs = obs[obs != 0] # Remove all the zeros from the sequence
  print('SeqNumber', i, ':',end=' ')
  print(viterbi(h_or_c, obs, prior_prob, stop_prob, transition_prob, emission_prob))
  i+=1

SeqNumber 1 : ['H', 'H', 'H', 'H', 'H']
SeqNumber 2 : ['H', 'H', 'H', 'H']
SeqNumber 3 : ['C', 'C', 'C', 'C', 'C']
SeqNumber 4 : ['C', 'C', 'C']
SeqNumber 5 : ['C', 'C', 'C', 'H', 'H']
SeqNumber 6 : ['C', 'C', 'C', 'C']
SeqNumber 7 : ['H', 'H', 'H']
SeqNumber 8 : ['C', 'C', 'C', 'C']
SeqNumber 9 : ['C', 'H', 'H', 'H', 'H']
SeqNumber 10 : ['C', 'C', 'C']


**The viterbi is suggesting that having 3 ice creams is defintely a hot day and having only 1 ice cream is a cold day. And when it comes to have 2 ice creams, it really depends on the day after. If the day after had 3 ice creams, then the day which had 2 ice creams is likely to be a hot day. If the day after had 1 ice cream, then the day which had 2 ice creams is more likely to be a cold day.**


**Part 2 (50 points):**

Using the same network, implement likelihood (weighting) sampling for approximate inference.  For any test sequence, sample complete sequences of the hidden states n times, where n can range from 10 to 100000 samples. The goal is to approximate the likelihood of all possible sequences.



In [ ]:
import copy
normalized_transition = copy.deepcopy(transition_prob)
display(normalized_transition)
for y_weather in h_or_c:
  prob_sum = 0
  for t_weather in h_or_c:
    prob_sum += normalized_transition[t_weather][y_weather]
  for t_weather in h_or_c:
    normalized_transition[t_weather][y_weather] = normalized_transition[t_weather][y_weather]/prob_sum
display(normalized_transition)

{'C': {'C': 0.87, 'H': 0.47}, 'H': {'C': 0.07, 'H': 0.47}}

{'C': {'C': 0.9255319148936171, 'H': 0.5},
 'H': {'C': 0.07446808510638299, 'H': 0.5}}

In [ ]:
import random
def likelihood(obs, prior_prob, transition_prob, emission_prob, iterations):
  sample_seq = {}
  for _ in range(iterations):
    random_prob = random.random()
    weather_seq = []
    weather = ''
    if random_prob < prior_prob['C']:
      weather = 'C'
    else:
      weather = 'H'
    weather_seq.append(weather)
    weight = emission_prob[weather][obs[0]]
    for i in range(1, len(obs)):
      prob_t_given_y = transition_prob['C'][weather]
      random_prob = random.random()
      if random_prob < prob_t_given_y:
        weather = 'C'
      else:
        weather = 'H'
      weather_seq.append(weather)
      weight *= emission_prob[weather][obs[i]]
    if tuple(weather_seq) in sample_seq:
      sample_seq[tuple(weather_seq)] += weight
    else:
      sample_seq[tuple(weather_seq)] = weight

  key_list = list(sample_seq.keys())
  val_list = list(sample_seq.values())
  max_weight = max(val_list) # Find the max weight
  position = val_list.index(max_weight)
  most_likely_sequence = key_list[position]

  return most_likely_sequence

In [ ]:
i = 1
print('Generate 10 samples: ')
for obs in observation_seq:
  obs = obs[obs != 0] # Remove all the zeros from the sequence
  print('SeqNumber', i, ':',end=' ')
  print(likelihood(obs, prior_prob, transition_prob, emission_prob, 10))
  i+=1

Generate 10 samples: 
SeqNumber 1 : ('H', 'H', 'H', 'H', 'C')
SeqNumber 2 : ('H', 'H', 'H', 'H')
SeqNumber 3 : ('C', 'C', 'C', 'C', 'C')
SeqNumber 4 : ('H', 'C', 'C')
SeqNumber 5 : ('C', 'C', 'C', 'C', 'H')
SeqNumber 6 : ('C', 'C', 'C', 'C')
SeqNumber 7 : ('H', 'H', 'H')
SeqNumber 8 : ('H', 'H', 'C', 'C')
SeqNumber 9 : ('C', 'C', 'C', 'H', 'H')
SeqNumber 10 : ('C', 'C', 'C')


In [ ]:
i = 1
print('Generate 100 samples: ')
for obs in observation_seq:
  obs = obs[obs != 0] # Remove all the zeros from the sequence
  print('SeqNumber', i, ':',end=' ')
  print(likelihood(obs, prior_prob, transition_prob, emission_prob, 100))
  i+=1

Generate 100 samples: 
SeqNumber 1 : ('H', 'H', 'H', 'H', 'H')
SeqNumber 2 : ('H', 'H', 'H', 'H')
SeqNumber 3 : ('C', 'C', 'C', 'C', 'C')
SeqNumber 4 : ('H', 'C', 'C')
SeqNumber 5 : ('C', 'C', 'C', 'C', 'C')
SeqNumber 6 : ('C', 'C', 'C', 'C')
SeqNumber 7 : ('H', 'H', 'H')
SeqNumber 8 : ('H', 'H', 'C', 'C')
SeqNumber 9 : ('C', 'H', 'H', 'H', 'H')
SeqNumber 10 : ('C', 'C', 'C')


In [ ]:
i = 1
print('Generate 1000 samples: ')
for obs in observation_seq:
  obs = obs[obs != 0] # Remove all the zeros from the sequence
  print('SeqNumber', i, ':',end=' ')
  print(likelihood(obs, prior_prob, transition_prob, emission_prob, 1000))
  i+=1

Generate 1000 samples: 
SeqNumber 1 : ('H', 'H', 'H', 'H', 'H')
SeqNumber 2 : ('H', 'H', 'H', 'H')
SeqNumber 3 : ('H', 'C', 'C', 'C', 'C')
SeqNumber 4 : ('H', 'C', 'C')
SeqNumber 5 : ('C', 'C', 'C', 'C', 'C')
SeqNumber 6 : ('C', 'C', 'C', 'C')
SeqNumber 7 : ('H', 'H', 'H')
SeqNumber 8 : ('H', 'H', 'C', 'C')
SeqNumber 9 : ('C', 'C', 'H', 'H', 'H')
SeqNumber 10 : ('C', 'C', 'C')


In [ ]:
i = 1
print('Generate 10000 samples: ')
for obs in observation_seq:
  obs = obs[obs != 0] # Remove all the zeros from the sequence
  print('SeqNumber', i, ':',end=' ')
  print(likelihood(obs, prior_prob, transition_prob, emission_prob, 10000))
  i+=1

Generate 10000 samples: 
SeqNumber 1 : ('H', 'H', 'H', 'H', 'H')
SeqNumber 2 : ('H', 'H', 'H', 'H')
SeqNumber 3 : ('H', 'C', 'C', 'C', 'C')
SeqNumber 4 : ('H', 'C', 'C')
SeqNumber 5 : ('C', 'C', 'C', 'C', 'C')
SeqNumber 6 : ('C', 'C', 'C', 'C')
SeqNumber 7 : ('H', 'H', 'H')
SeqNumber 8 : ('H', 'H', 'C', 'C')
SeqNumber 9 : ('C', 'H', 'H', 'H', 'H')
SeqNumber 10 : ('C', 'C', 'C')


In [ ]:
i = 1
print('Generate 100000 samples: ')
for obs in observation_seq:
  obs = obs[obs != 0] # Remove all the zeros from the sequence
  print('SeqNumber', i, ':',end=' ')
  print(likelihood(obs, prior_prob, transition_prob, emission_prob, 100000))
  i+=1

Generate 100000 samples: 
SeqNumber 1 : ('H', 'H', 'H', 'H', 'H')
SeqNumber 2 : ('H', 'H', 'H', 'H')
SeqNumber 3 : ('H', 'C', 'C', 'C', 'C')
SeqNumber 4 : ('H', 'C', 'C')
SeqNumber 5 : ('C', 'C', 'C', 'C', 'C')
SeqNumber 6 : ('C', 'C', 'C', 'C')
SeqNumber 7 : ('H', 'H', 'H')
SeqNumber 8 : ('H', 'H', 'C', 'C')
SeqNumber 9 : ('C', 'H', 'H', 'H', 'H')
SeqNumber 10 : ('C', 'C', 'C')


**As the number of samples increases, the likelihood is getting more and more acturate comparing with the result of viterbi. But the overhead of this likelihood is increasing dramatically as the size of the sample increases.**
